In [1]:
from vibegrad import Tensor

In [2]:
a = Tensor(2)
b = Tensor(4)

In [3]:
c = a*b
c.backward()
c.grad

1

In [4]:
b.grad, a.grad

(2.0, 4.0)

In [5]:
c.zero_grad()
a.zero_grad()
b.zero_grad()

In [6]:
from vibegrad.nn import Linear, Sequential

In [7]:
a = Linear(1, 1)

In [8]:
import numpy as np
np.random.uniform((1,1))

array([1., 1.])

In [9]:
a(Tensor([1]))

Tensor(data=[0.0104803])

In [10]:
a.weight, a.bias

(Tensor(data=[[0.0104803]]), array([0.]))

In [11]:
import torch
from torch.nn import Linear as tLinear

In [12]:
a = tLinear(1,1)

In [13]:
a(torch.tensor([1.0]))

tensor([-1.1977], grad_fn=<ViewBackward0>)

In [14]:
a.weight, a.bias

(Parameter containing:
 tensor([[-0.9004]], requires_grad=True),
 Parameter containing:
 tensor([-0.2973], requires_grad=True))

In [15]:
from vibegrad.nn import *

In [16]:
model = Sequential([
    Linear(256, 300),
    ReLU(),
    Linear(300, 300),
    ReLU(),
    Linear(300, 10),
    Sigmoid()
])
model

In [17]:
model.parameters

'Linear(256, 300, bias=True) <vibegrad.nn.activations.ReLU object at 0x70f16f35bd90>Linear(300, 300, bias=True) <vibegrad.nn.activations.ReLU object at 0x70f16f35a1d0>Linear(300, 10, bias=True) <vibegrad.nn.activations.Sigmoid object at 0x70f16f35b9d0>'

In [18]:
model.total_params()

170410

In [19]:
X = np.random.uniform(-1, 1, size=(100, 256))
y = np.random.choice([0, 1], size=(100, 10), p=[0.5, 0.5])
X.shape
X = Tensor(X)
y = Tensor(y) 
y.data[:5]

array([[0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 0]])

In [20]:
out = model(X)
out.data.shape, y.data.shape

((100, 10), (100, 10))

In [21]:
loss_fn = BCELoss(reduction="mean")
print(out)
loss = loss_fn(out, y)
print(loss)

Tensor(data=[[0.52371102 0.52212771 0.52391284 0.52243536 0.52238915 0.52378274
  0.52223559 0.52238869 0.52334069 0.52391554]
 [0.74021388 0.72805077 0.74175655 0.72998293 0.72517927 0.73907045
  0.72589462 0.72904875 0.73542084 0.74051033]
 [0.72459405 0.71512509 0.72736076 0.7169332  0.71062387 0.72533446
  0.71100673 0.715234   0.72317662 0.72634784]
 [0.99758474 0.99679201 0.99779424 0.99684705 0.99638575 0.99762022
  0.99637045 0.99685512 0.99737367 0.99768255]
 [0.9997031  0.9995442  0.99973032 0.99957144 0.9994689  0.9997062
  0.99948006 0.99956429 0.9996605  0.99971294]
 [0.99902275 0.99860037 0.9991117  0.99864446 0.99837228 0.99900944
  0.99841368 0.99862442 0.99887627 0.9990477 ]
 [0.89751351 0.8856383  0.89912556 0.88727056 0.88091905 0.89704192
  0.88215501 0.8869923  0.89373865 0.89730334]
 [0.99982323 0.9997218  0.99984168 0.99973662 0.99967246 0.99982638
  0.99967714 0.99973606 0.9997948  0.99982937]
 [1.         1.         1.         1.         1.         1.
  1.     

In [22]:
import torch
out = torch.tensor(out.data)
y = torch.tensor(y.data)
loss_fn_torch = torch.nn.BCELoss()
y = y.float()
out = out.float()
loss_fn_torch(out, y)

tensor(13.3998)

In [23]:
a = Tensor([[0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0]])

for ding in a.data:
    for dong in ding:
        if dong != 1 and dong != 0:
            raise ValueError("ding dong")    

In [32]:
prediction = Tensor(np.array([0.9, 0.4, 0.7, 0.2]))
target = Tensor(np.array([1, 0, 1, 0]))

loss_fn = BCELoss()
loss = loss_fn(prediction, target)
print(loss)
loss.backward()

0.2990011586691898


AttributeError: 'numpy.float64' object has no attribute 'backward'

In [33]:
prediction = torch.tensor(np.array([0.9, 0.4, 0.7, 0.2]))
target = torch.tensor(np.array([1,0,1,0]), dtype=torch.double)
loss_fn_torch = BCELoss()
loss_fn_torch(prediction, target)
loss_fn_torch.backward()

TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype = None)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None)
 * (tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None)


In [23]:
model = torch.nn.Sequential()
model.append(torch.nn.Linear(256, 300))
model.append(torch.nn.Linear(300,300))
model.append(torch.nn.Linear(300,10))

Sequential(
  (0): Linear(in_features=256, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=300, bias=True)
  (2): Linear(in_features=300, out_features=10, bias=True)
)

In [24]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of parameters: {total_params}")

Total number of parameters: 170410


In [25]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=256, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=300, bias=True)
  (2): Linear(in_features=300, out_features=10, bias=True)
)>

In [26]:
import vibegrad.nn as nn

In [27]:
nn.relu(Tensor(2.0))

Tensor(data=2.0)